We're finally ready to start start downloading the blockchain.

Read the wiki's explanation of headers-first and 

Initial block download requires we do a few things:
1. Execute the version handshake
2. Interpret Bitcoin network messages and deserialize / serialize message payloads.
3. Persist the data we receive (sqlite for now)
4. Handle multiple peer connections concurrently (threading)

In the past four lessons we've learned to do each of these things



This is CBlockLocator being built: https://github.com/bitcoin/bitcoin/blob/master/src/chain.cpp#L23

# Requesting Headers

Since 2014 Bitcoin Core has used a ["headers-first" algorithm](https://en.bitcoin.it/wiki/Bitcoin_Core_0.11_(ch_5):_Initial_Block_Download#.22Headers_First.22_mode) for initial block download. We will start our attempt to implement this same algorithm for our initial block download program.

This algorithm first downloads and validates headers for every block in the blockchain. What is the difference betweeen blocks and block-headers? Blocks contain every transaction within the block, whereas headers do now. That's the only difference. If you look at the [Bitcoin Core source code](https://github.com/bitcoin/bitcoin/blob/master/src/primitives/block.h) where both interfaces are defined, you will notice that `CBlock` actually subclasses `CBlockHeader`.

So this algorithm basically assembles the entire blockchain sans transactions, then goes back and fills the transactions in.

So our first task is clear -- we need to request headers from our peers. This can be accomplished with the `getheaders` message. Inspecting its table in the [protocol documentation](https://en.bitcoin.it/wiki/Protocol_documentation#getheaders) we see that the `version` (int), `hash count` (var int), `hash_stop` (bytes of a sha256 hash, just like the `Packet.checksum`) are of types we're familiar with. 

But then there is `block locator hashes`. This will be trickier. It is a concatenation of raw byte hashes like the `hash_stop` attribute, but which hashes are relevant for this field? I was stumped for a good while when I first tried to create one of these. [This page from the Bitcoin wiki](https://en.bitcoin.it/wiki/Block_chain_download#Locator) seems to say you 
* add your 10 most recent hashes
* define a `step = 1` variable
* begin iterating with every iteration doubling `step`
* start iterating on 11th hash (step=1)
* stop iterating once `step` becomes larger than the number of blocks in your chain
* within each loop of you add the block that is `step` blocks from the tip of your chain
* after iteration completes you append the genesis block's header

The structure you are left with looks like this:
* header of block with greatest height
* header of block with 2nd greatest height
* ...
* header of block with 10th greatest height
* header of block with 11th greatest height
* header of block with 12th greatest height
* header of block with 14th greatest height
* header of block with 16th greatest height
* ...
* header of block with `10+step` greatest height
* ...
* header of genesis block

The purpose of this structure is to assist our peer in their search to find the block of greatest height that we still agree upon. Often times this will be the tip of our chain, but sometimes it won't. In such a case it would usually be our 2nd most recent block, and so on back into the history of our chain -- with older blocks having lower and lower probabilities of being the most recent agreed upon block.

In [ ]:
%load_ext autoreload
%autoreload 2

from ibd.three.complete import *
from ibd.three.handshake import handshake

In [ ]:
class GetHeadersMessage:

    command = b"getheaders"

    def __init__(self, locator, hashstop=0):
        self.locator = locator
        self.hashstop = hashstop

    def to_bytes(self):
        msg = self.locator.to_bytes()
        msg += int_to_bytes(self.hashstop, 32)
        return msg

In [ ]:
class BlockLocator:
    def __init__(self, hashes, version):
        self.hashes = hashes
        self.version = version

    def to_bytes(self):
        msg = int_to_bytes(self.version, 4)
        msg += int_to_var_int(len(self.hashes))
        for hash_ in self.hashes:
            msg += int_to_bytes(hash_, 32)
        return msg

In [ ]:
VERSION = 70015
genesis_hash = int("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f", 16)

def construct_block_locator(all_hashes):
    locator_hashes = []
    height = len(all_hashes) - 1
    step = 1

    while height >= 0:
        # every iteration adds header at `height` and decrements `step`
        header = all_hashes[height]
        locator_hashes.append(header)
        height -= step
        
        # `step` starts doubling after the 11th hash
        if len(locator_hashes) > 10:
            step *= 2
    
    # make sure we have the genesis hash
    if not locator_hashes.index(genesis_hash):
        locator_hashes.append(genesis_hash)

    return BlockLocator(hashes=locator_hashes, version=VERSION)

In [ ]:
header_hashes = [genesis_hash]

def send_getheaders(sock):
    locator = construct_block_locator(header_hashes)
    getheaders = GetHeadersMessage(locator)
    packet = Packet(getheaders.command, getheaders.to_bytes())
    sock.send(packet.to_bytes())

def initial_block_download():
    address = ("91.221.70.137", 8333)
    sock = handshake(address, log=False)
    # comment this line out and we don't get any headers
    send_getheaders(sock)
    while True:
        packet = Packet.from_socket(sock)
        print(f"received {packet.command}")

# Interpreting Block Headers

We can now request and receive `header` messages -- but we don't know how to interpret them yet. Here are 2 classes that will allow us to do just that:

In [ ]:
class HeadersMessage:

    command = b"headers"

    def __init__(self, count, headers):
        self.count = count
        self.headers = headers

    @classmethod
    def from_bytes(cls, b):
        s = io.BytesIO(b)
        count = read_var_int(s)
        headers = []
        for _ in range(count):
            header = BlockHeader.from_stream(s)
            headers.append(header)
        return cls(count, headers)

    def __repr__(self):
        return f"<HeadersMessage #{len(self.headers)}>"


def double_sha256(b):
    first_round = hashlib.sha256(b).digest()
    second_round = hashlib.sha256(first_round).digest()
    return second_round

    
class BlockHeader:
    def __init__(
        self, version, prev_block, merkle_root, timestamp, bits, nonce, txn_count
    ):
        self.version = version
        self.prev_block = prev_block
        self.merkle_root = merkle_root
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = nonce
        self.txn_count = txn_count

    @classmethod
    def from_stream(cls, s):
        version = read_int(s, 4)
        prev_block = read_int(s, 32)
        merkle_root = read_int(s, 32)
        timestamp = read_int(s, 4)
        bits = s.read(4)
        nonce = s.read(4)
        txn_count = read_var_int(s)
        return cls(version, prev_block, merkle_root, timestamp, bits, nonce, txn_count)

    def to_bytes(self):
        result = int_to_bytes(self.version, 4)
        result += int_to_bytes(self.prev_block, 32)
        result += int_to_bytes(self.merkle_root, 32)
        result += int_to_bytes(self.timestamp, 4)
        result += self.bits
        result += self.nonce
        return result

    def hash(self):
        s = self.to_bytes()
        sha = double_sha256(s)
        return sha[::-1]  # little endian

    def pow(self):
        s = self.to_bytes()
        sha = double_sha256(s)
        return bytes_to_int(sha)

    def target(self):
        # last byte is exponent
        exponent = self.bits[-1]
        # the first three bytes are the coefficient in little endian
        coefficient = bytes_to_int(self.bits[:-1])
        # the formula is:
        # coefficient * 2**(8*(exponent-3))
        return coefficient * 2 ** (8 * (exponent - 3))

    def check_pow(self):
        return self.pow() < self.target()

    def pretty(self):
        hx = hex(self.pow())[2:]  # remove "0x" prefix
        sigfigs = len(hx)
        padding = "0" * (64 - sigfigs)
        return padding + hx

    def __str__(self):
        headers = ["BlockHeader", ""]
        attrs = [
            "version",
            "prev_block",
            "merkle_root",
            "timestamp",
            "bits",
            "nonce",
            "txn_count",
        ]
        rows = [[attr, fmt(getattr(self, attr))] for attr in attrs]
        rows = [["hash", self.pretty()]] + rows
#         import pdb; pdb.set_trace()
        return tabulate(rows, headers, tablefmt="grid")


In [ ]:
def initial_block_download():
    address = ("91.221.70.137", 8333)
    sock = handshake(address, log=False)
    # comment this line out and we don't get any headers
    send_getheaders(sock)
    while True:
        packet = Packet.from_socket(sock)
        if packet.command == b"headers":
            header_message = HeadersMessage.from_bytes(packet.payload)
            for header in header_message.headers:
                print(header)
        else:
            print(f"received {packet.command}")

In [ ]:
initial_block_download()

# Ordering and Saving Headers

Now that we have a way to download and interpret block headers, let's save then in an ordered list -- a "chain" if you will ...

In [ ]:
def save_header_hashes(block_headers):
    for header in block_headers:
        # we add it to the header_hashes if prev_block is our current tip
        if header.prev_block == header_hashes[-1]:
            header_hashes.append(header.pow())
        else:
            raise RuntimeError("received out-of-order block")
            

def handle_headers_packet(packet, sock):
    headers_message = HeadersMessage.from_bytes(packet.payload)
    block_headers = headers_message.headers
    print(f"{len(block_headers)} new headers")
    save_header_hashes(block_headers)
    print(f"We now have {len(header_hashes)} headers")


def initial_block_download():
    address = ("91.221.70.137", 8333)
    sock = handshake(address, log=False)
    # comment this line out and we don't get any headers
    send_getheaders(sock)
    while True:
        packet = Packet.from_socket(sock)
        if packet.command == b"headers":
            handle_headers_packet(packet, sock)
        else:
            print(f"received {packet.command}")

In [ ]:
header_hashes = [genesis_hash]
initial_block_download()

And let me assure you, if we replied to the `headers` message with another `getheaders` message containing an updated "block locator", we'd get 2000 more headers. But I think that's enough for now.

# Downloading Blocks

Now we can request, receive, interpret and save the headers that we receive let's try to download the corresponding blocks.

In [ ]:
class Tx:
    def __init__(self, version, tx_ins, tx_outs, locktime, testnet=False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.locktime = locktime
        self.testnet = testnet

    @classmethod
    def from_stream(cls, s):
        """Takes a byte stream and from_streams the transaction at the start
        return a Tx object
        """
        # s.read(n) will return n bytes
        # version has 4 bytes, little-endian, interpret as int
        version = bytes_to_int(s.read(4))
        # num_inputs is a varint, use read_var_int(s)
        num_inputs = read_var_int(s)
        # each input needs parsing
        inputs = []
        for _ in range(num_inputs):
            inputs.append(TxIn.from_stream(s))
        # num_outputs is a varint, use read_var_int(s)
        num_outputs = read_var_int(s)
        # each output needs parsing
        outputs = []
        for _ in range(num_outputs):
            outputs.append(TxOut.from_stream(s))
        # locktime is 4 bytes, little-endian
        locktime = bytes_to_int(s.read(4))
        # return an instance of the class (cls(...))
        return cls(version, inputs, outputs, locktime)

    def __repr__(self):
        return "<Tx version: {} ntx_ins: {} tx_outs: {} nlocktime: {}>".format(
            self.version,
            ",".join([repr(t) for t in self.tx_ins]),
            ",".join([repr(t) for t in self.tx_outs]),
            self.locktime,
        )


class TxIn:
    def __init__(self, prev_tx, prev_index, script_sig, sequence):
        self.prev_tx = prev_tx
        self.prev_index = prev_index
        self.script_sig = script_sig  # TODO from_stream it
        self.sequence = sequence

    def __repr__(self):
        return "<TxIn {}:{}>".format(self.prev_tx.hex(), self.prev_index)

    @classmethod
    def from_stream(cls, s):
        """Takes a byte stream and from_streams the tx_input at the start
        return a TxIn object
        """
        # s.read(n) will return n bytes
        # prev_tx is 32 bytes, little endian
        prev_tx = s.read(32)[::-1]
        # prev_index is 4 bytes, little endian, interpret as int
        prev_index = bytes_to_int(s.read(4))
        # script_sig is a variable field (length followed by the data)
        # get the length by using read_var_int(s)
        script_sig_length = read_var_int(s)
        script_sig = s.read(script_sig_length)
        # sequence is 4 bytes, little-endian, interpret as int
        sequence = bytes_to_int(s.read(4))
        # return an instance of the class (cls(...))
        return cls(prev_tx, prev_index, script_sig, sequence)


class TxOut:
    def __init__(self, amount, script_pubkey):
        self.amount = amount
        self.script_pubkey = script_pubkey  # TODO from_stream it

    def __repr__(self):
        return "<TxOut {}:{}>".format(self.amount, self.script_pubkey)

    @classmethod
    def from_stream(cls, s):
        """Takes a byte stream and from_streams the tx_output at the start
        return a TxOut object
        """
        # s.read(n) will return n bytes
        # amount is 8 bytes, little endian, interpret as int
        amount = bytes_to_int(s.read(8))
        # script_pubkey is a variable field (length followed by the data)
        # get the length by using read_var_int(s)
        script_pubkey_length = read_var_int(s)
        script_pubkey = s.read(script_pubkey_length)
        # return an instance of the class (cls(...))
        return cls(amount, script_pubkey)


class Block(BlockHeader):
    def __init__(
        self, version, prev_block, merkle_root, timestamp, bits, nonce, txn_count, txns
    ):
        super().__init__(
            version, prev_block, merkle_root, timestamp, bits, nonce, txn_count
        )
        self.txns = txns

    @classmethod
    def from_stream(cls, s):
        version = read_int(s, 4)
        prev_block = read_int(s, 32)
        merkle_root = read_int(s, 32)
        timestamp = read_int(s, 4)
        bits = s.read(4)
        nonce = s.read(4)
        txn_count = read_var_int(s)
        txns = [Tx.from_stream(s) for _ in range(txn_count)]
        return cls(
            version, prev_block, merkle_root, timestamp, bits, nonce, txn_count, txns
        )

    def __repr__(self):
        return f"<Block {self.pretty()} >"

In [ ]:
class InventoryItem:
    def __init__(self, type_, hash_):
        self.type = type_
        self.hash = hash_

    @classmethod
    def from_stream(cls, s):
        type_ = bytes_to_int(s.read(4))
        hash_ = s.read(32)
        return cls(type_, hash_)

    def to_bytes(self):
        msg = b""
        msg += int_to_bytes(self.type, 4)
        msg += self.hash
        return msg

    def __repr__(self):
        return f"<InvItem {inv_map[self.type]} {self.hash}>"


class GetData:
    command = b"getdata"

    def __init__(self, items=None):
        if items is None:
            self.items = []
        else:
            self.items = items

    def to_bytes(self):
        msg = int_to_var_int(len(self.items))
        for item in self.items:
            msg += item.to_bytes()
        return msg

    def __repr__(self):
        return f"<Getdata {repr(self.inv)}>"


In [ ]:
def request_blocks(sock):
    items = [InventoryItem(2, int_to_bytes(hash_, 32)) 
             for hash_ in header_hashes]
    getdata = GetData(items=items)
    packet = Packet(getdata.command, getdata.to_bytes())
    sock.send(packet.to_bytes())
    
def handle_headers_packet(packet, sock):
    headers_message = HeadersMessage.from_bytes(packet.payload)
    block_headers = headers_message.headers
    print(f"{len(block_headers)} new headers")
    save_header_hashes(block_headers)
    print(f"We now have {len(header_hashes)} headers")
    request_blocks(sock)
    
def handle_block_packet(packet, sock):
    block = Block.from_stream(io.BytesIO(packet.payload))
    for txn in block.txns:
        print(txn)
    
def handle_packet(packet, sock):
    command_to_handler = {
        b"headers": handle_headers_packet,
        b"block": handle_block_packet,
    }
    handler = command_to_handler.get(packet.command)
    if handler:
        print(f'handling "{packet.command}"')
        handler(packet, sock)
    else:
        print(f'discarding "{packet.command}"')

        
def initial_block_download():
    address = ("91.221.70.137", 8333)
    sock = handshake(address, log=False)
    # comment this line out and we don't get any headers
    send_getheaders(sock)
    while True:
        packet = Packet.from_socket(sock)
        handle_packet(packet, sock)

In [ ]:
header_hashes = [genesis_hash]
initial_block_download()

# Next Time

* Validate transactions in each block
    * Evaluate scripts
* Store valid blocks in a "block chain"
* Maintain UTXO database
* Download from multiple peers